# **Feladatleírás**

A projekt során egy felügyelt gépi tanulási problémát oldunk meg egy általunk tetszőlegesen választott adatbázison. A munkához egy banki adatbázist választottunk, amiben a bank az ügyfeleiről tárol el különféle adatokat. Ezek alapján azt szeretnénk megjósolni, hogy egy új ügyfél a jövőben fog-e betéti számlát nyitni.

# Importok

In [1]:
import os
from zipfile import ZipFile
import urllib.request
import pandas as pd
import numpy as np
from sklearn import datasets
from sklearn import preprocessing
from sklearn.preprocessing import MinMaxScaler
from sklearn.feature_selection import SelectKBest, chi2

#Paraméterek

In [2]:
str_labels = ['job', 'marital', 'education', 'default', 'housing', 'loan']
int_labels = ['age', 'balance']
labels = str_labels + int_labels

empty_drop = True

train_size = 0.7
dev_size = 0.15
test_size = 1 - train_size - dev_size

# **Dataset**

Az adatokat egy linkből importáltuk be. Az így elért adatokat kicsomagoltuk és betöltöttük.

In [3]:
url = 'https://archive.ics.uci.edu/ml/machine-learning-databases/00222/bank.zip'
urllib.request.urlretrieve(url, './bank.zip') 
with ZipFile('bank.zip', "r") as zipObj:
    zipObj.extractall()

bankdata_full = pd.read_csv("./bank-full.csv", delimiter=';')
# Tesztelesbol kiirjuk a beolvasott .csv fajl egyes reszeit
print("Adatrekordok szama: " + str(len(bankdata_full.index)))
print(np.sort((bankdata_full['age'].unique())))

Adatrekordok szama: 45211
[18 19 20 21 22 23 24 25 26 27 28 29 30 31 32 33 34 35 36 37 38 39 40 41
 42 43 44 45 46 47 48 49 50 51 52 53 54 55 56 57 58 59 60 61 62 63 64 65
 66 67 68 69 70 71 72 73 74 75 76 77 78 79 80 81 82 83 84 85 86 87 88 89
 90 92 93 94 95]


# **Data processing (MK - I)**

Megnéztük, hogy mennyi olyan sorunk van az adathalmazban, amiben van üres cella.

In [4]:
for label in labels:
    i = 0
    for l in bankdata_full[label]:
        if l == "unknown":
            i += 1
    print(label + ": " + str(i))

job: 288
marital: 0
education: 1857
default: 0
housing: 0
loan: 0
age: 0
balance: 0


Majd ezeket kitöröltük.

In [5]:
if empty_drop:
    for label in labels:
        for l in bankdata_full[label]:
            if l == "unknown":
                bankdata_dropped = bankdata_full.drop(bankdata_full.loc[bankdata_full[label] == "unknown"].index, inplace=False)
    print(bankdata_dropped)
else:
    pass
    # TODO: eldobás helyett valami más módszer az üres cellát tartalmazó sorok kezelésére

       age           job   marital  education default  balance housing loan  \
0       58    management   married   tertiary      no     2143     yes   no   
1       44    technician    single  secondary      no       29     yes   no   
2       33  entrepreneur   married  secondary      no        2     yes  yes   
5       35    management   married   tertiary      no      231     yes   no   
6       28    management    single   tertiary      no      447     yes  yes   
...    ...           ...       ...        ...     ...      ...     ...  ...   
45206   51    technician   married   tertiary      no      825      no   no   
45207   71       retired  divorced    primary      no     1729      no   no   
45208   72       retired   married  secondary      no     5715      no   no   
45209   57   blue-collar   married  secondary      no      668      no   no   
45210   37  entrepreneur   married  secondary      no     2971      no   no   

         contact  day month  duration  campaign  pd

**Ordial Encoder**

In [6]:
encoded_str = bankdata_dropped[str_labels]

oe = preprocessing.OrdinalEncoder()
encoded_str[str_labels] = oe.fit_transform(encoded_str[str_labels])

print(encoded_str)

       job  marital  education  default  housing  loan
0      4.0      1.0        2.0      0.0      1.0   0.0
1      9.0      2.0        1.0      0.0      1.0   0.0
2      2.0      1.0        1.0      0.0      1.0   1.0
5      4.0      1.0        2.0      0.0      1.0   0.0
6      4.0      2.0        2.0      0.0      1.0   1.0
...    ...      ...        ...      ...      ...   ...
45206  9.0      1.0        2.0      0.0      0.0   0.0
45207  5.0      0.0        0.0      0.0      0.0   0.0
45208  5.0      1.0        1.0      0.0      0.0   0.0
45209  1.0      1.0        1.0      0.0      0.0   0.0
45210  2.0      1.0        1.0      0.0      0.0   0.0

[43354 rows x 6 columns]


/usr/local/lib/python3.7/dist-packages/pandas/core/frame.py:3678: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self[col] = igetitem(value, i)


**MinMax Scaler**

In [7]:
min_max_scaler = preprocessing.MinMaxScaler()
encoded_str[str_labels] = min_max_scaler.fit_transform(encoded_str[str_labels])

print(encoded_str)

            job  marital  education  default  housing  loan
0      0.363636      0.5        1.0      0.0      1.0   0.0
1      0.818182      1.0        0.5      0.0      1.0   0.0
2      0.181818      0.5        0.5      0.0      1.0   1.0
5      0.363636      0.5        1.0      0.0      1.0   0.0
6      0.363636      1.0        1.0      0.0      1.0   1.0
...         ...      ...        ...      ...      ...   ...
45206  0.818182      0.5        1.0      0.0      0.0   0.0
45207  0.454545      0.0        0.0      0.0      0.0   0.0
45208  0.454545      0.5        0.5      0.0      0.0   0.0
45209  0.090909      0.5        0.5      0.0      0.0   0.0
45210  0.181818      0.5        0.5      0.0      0.0   0.0

[43354 rows x 6 columns]


/usr/local/lib/python3.7/dist-packages/pandas/core/frame.py:3678: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self[col] = igetitem(value, i)


**Standard Scaler**

In [8]:
encoded_int = bankdata_dropped[int_labels]

ss = preprocessing.StandardScaler()
encoded_int[int_labels] = ss.fit_transform(encoded_int[int_labels])

print(encoded_int)

            age   balance
0      1.636763  0.259146
1      0.305821 -0.436276
2     -0.739919 -0.445158
5     -0.549785 -0.369826
6     -1.215256 -0.298770
...         ...       ...
45206  0.971292 -0.174423
45207  2.872638  0.122957
45208  2.967705  1.434192
45209  1.541696 -0.226070
45210 -0.359650  0.531525

[43354 rows x 2 columns]


/usr/local/lib/python3.7/dist-packages/pandas/core/frame.py:3678: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self[col] = igetitem(value, i)


**Ellenőrzésképp átlag és szórás kiszámolása**

In [9]:
print(np.mean(encoded_int['age'],axis=0))
print(np.std(encoded_int['age'],axis=0))

-1.1595856186710912e-15
1.0000000000000047


**Elkészült adatoszlopok összefűzése**

In [10]:
encoded_full = pd.concat([encoded_str, encoded_int], axis=1)
print(encoded_full)

            job  marital  education  default  housing  loan       age  \
0      0.363636      0.5        1.0      0.0      1.0   0.0  1.636763   
1      0.818182      1.0        0.5      0.0      1.0   0.0  0.305821   
2      0.181818      0.5        0.5      0.0      1.0   1.0 -0.739919   
5      0.363636      0.5        1.0      0.0      1.0   0.0 -0.549785   
6      0.363636      1.0        1.0      0.0      1.0   1.0 -1.215256   
...         ...      ...        ...      ...      ...   ...       ...   
45206  0.818182      0.5        1.0      0.0      0.0   0.0  0.971292   
45207  0.454545      0.0        0.0      0.0      0.0   0.0  2.872638   
45208  0.454545      0.5        0.5      0.0      0.0   0.0  2.967705   
45209  0.090909      0.5        0.5      0.0      0.0   0.0  1.541696   
45210  0.181818      0.5        0.5      0.0      0.0   0.0 -0.359650   

        balance  
0      0.259146  
1     -0.436276  
2     -0.445158  
5     -0.369826  
6     -0.298770  
...         ...

**Adathalmaz felosztása**

In [11]:
train, dev, test = np.split(encoded_full.sample(frac=1, random_state=123456),
                            [int(train_size*len(encoded_full)), int((train_size+dev_size)*len(encoded_full))])
print("Train size: " + str(train.size // 8))
print("Dev size: " + str(dev.size // 8))
print("Test size: " + str(test.size // 8))

Train size: 30347
Dev size: 6503
Test size: 6504
